# Chapter 4: Full Robot Simulation

Welcome to the fourth chapter of our hands-on course! In this tutorial, you will focus on creating a complete robot control plan in pycram and gaining an understanding of fundamental concepts in robotic simulation.

### Learning Objectives
By the end of this chapter, you should be able to:
- Load and initialize a robot and environment for simulation.
- Plan and execute a robotic task that includes object detection, grasping, transporting, and placing.
- Understand key concepts like coordinate transformations, pose calculations, and error handling.

Let's get started!


## Step 1: Getting Started

In this step, we’ll set up the environment and load all necessary libraries. We will also initialize the simulation, creating a robot in a kitchen setting.

*Objective:* By the end of this step, you should have a fully initialized simulation environment and robot to work with.


In [ ]:
from pycram.external_interfaces.ik import request_ik
from pycram.plan_failures import IKError
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher, AxisMarkerPublisher, CostmapPublisher
from pycram.utils import _apply_ik
from pycram.worlds.bullet_world import BulletWorld
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.datastructures.enums import ObjectType, WorldMode, TorsoState
from pycram.datastructures.pose import Pose, Transform
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.object_descriptors.urdf import ObjectDescription
from pycram.world_concepts.world_object import Object
from pycram.datastructures.dataclasses import Color

extension = ObjectDescription.get_file_extension()

world = BulletWorld(WorldMode.DIRECT)
world.allow_publish_debug_poses = True
viz = VizMarkerPublisher()
tf = TFBroadcaster()

robot_name = "pr2"
robot = Object(robot_name, ObjectType.ROBOT, f"{robot_name}{extension}", pose=Pose([1, 2, 0]))

apartment = Object("apartment", ObjectType.ENVIRONMENT, f"apartment-small{extension}")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([0.5, 2.5, 1]))
milk.color = Color(0, 0, 1, 1)
milk_desig = BelieveObject(names=["milk"])
robot_desig = BelieveObject(names=[robot_name])
apartment_desig = BelieveObject(names=["apartment"])

## Step 2: Detecting the Milk

 Let's start where we left off with detecting the milk in the open fridge!


In [ ]:
with simulated_robot:
    poseHard = Pose([1.3, 2.7, 0], [0, 0, 1, 0])
    NavigateAction([poseHard]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([TorsoState.HIGH]).resolve().perform()
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    closed_location, opened_location = AccessingLocation(handle_desig=handle_desig.resolve(),
                                                         robot_desig=robot_desig.resolve()).resolve()
    OpenAction(object_designator_description=handle_desig, arms=[closed_location.arms[0]],
               start_goal_location=[closed_location, opened_location]).resolve().perform()
    
    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    obj_desig = DetectAction(milk_desig).resolve().perform()

# Step 3: Task Outline
Now, the remaining tasks are: picking up the milk, transporting it, and placing it back near the fridge. While this might seem simple to us, a robot control program must account for many complex details. 

For a robot to effectively pick up the milk, it needs to understand how to position and move its arm precisely, apply the right amount of force, and assess whether the milk is full, half-full, or empty, as each condition affects the grip strength and energy required. Additionally, it must know if the milk is open or closed, which influences how it should be handled and placed to prevent spills.

In this tutorial, we’ll focus on key concepts rather than delving into physical details. The pycram simulation environment, called Bullet World, is designed for rapid testing, allowing the robot to move faster than it would in a real-world setting. To save time, we also teleport the robot instead of having it navigate full paths. This has the advantage that we can quickly write and test plans for the robot.

Here’s what we will cover:

1. Parking both arms for initial positioning.
2. Determining the appropriate grasp based on the object type.
3. Finding a reachable pose for the robot to pick up the object with the chosen arm.
4. Navigating to the pick-up pose, performing the pick-up, and parking arms afterward.
5. Resolving a reachable location for placing the object.
6. Navigating to the placement location, performing the placement, and parking arms post-transport.

- **Question:** Why do you think it's important for the robot to calculate the approach direction when grasping an object?


## Step 4: Calculating Reachable Poses for Grasping

Now that we have detected the milk, we need to determine reachable poses for grasping it.

*Objective:* By the end of this step, you should be able to calculate a pose for the robot that allows it to reach and grasp the object.


#### The Grasp
Grasping is a challenging aspect of any robot control program. Depending on where the perception system detects the object, its orientation can vary significantly. Ideally, we want to instruct the robot to "pick up the milk from the front" or "from the top" in a way that’s easy for humans to specify. For example, regardless of how the milk is positioned in the fridge, the robot should always approach it from the fridge's front. However, if the robot is on the other side of the kitchen, this creates an orientation issue.

To address this, we developed the `calculate_object_face` function. This function takes an object as input and, using rotation matrices and the robot's position, calculates the correct orientation to approach the object based on its facing direction relative to the robot.


<details>

<summary>Click here for the code behind calculate_object:faces</summary>

```python
def calculate_object_faces(object):
    """
    Calculates the faces of an object relative to the robot based on orientation.

    This method determines the face of the object that is directed towards the robot,
    as well as the bottom face, by calculating vectors aligned with the robot's negative x-axis
    and negative z-axis in the object's frame.

    Args:
        object (Object): The object whose faces are to be calculated, with an accessible pose attribute.

    Returns:
        list: A list containing two Grasp Enums, where the first element is the face of the object facing the robot,
              and the second element is the top or bottom face of the object.
    """
    local_transformer = LocalTransformer()
    oTm = object.pose

    base_link = RobotDescription.current_robot_description.base_link
    marker = AxisMarkerPublisher()
    base_link_pose = object.world_object.world.robot.get_link_pose(base_link)

    marker.publish([base_link_pose])

    oTb = local_transformer.transform_pose(oTm, object.world_object.world.robot.get_link_tf_frame(base_link))
    orientation = oTb.orientation_as_list()

    rotation_matrix = R.from_quat([orientation[0], orientation[1], orientation[2], orientation[3]]).inv().as_matrix()

    robot_negative_x_vector = -rotation_matrix[:, 0]
    robot_negative_z_vector = -rotation_matrix[:, 2]

    facing_robot_face = calculate_vector_face(robot_negative_x_vector)
    bottom_face = calculate_vector_face(robot_negative_z_vector)

    return [facing_robot_face, bottom_face]

```
</details>

We’ll have identified the correct grasp based on the object’s orientation. Next, we’ll use `CostmapLocation` to determine a suitable position where the robot can stand to pick up the milk.

### Exercise: Finding a Suitable Pose with `CostmapLocation`

In this exercise, you’ll use `CostmapLocation` to identify a reachable location where the robot can pick up an object (in this case, the milk) using the correct grasp. Here’s a breakdown of what each parameter in `CostmapLocation` represents:

- **`target`**: The designator for the object you want the robot to pick up (e.g., `self.object_designator`).
- **`reachable_for`**: The robot or component that needs access to the target (use `robot_desig.resolve()` for the robot).
- **`reachable_arm`**: The specific arm that will perform the grasp (e.g., `self.arm`).
- **`used_grasps`**: The list of grasps that are suitable for the target object (e.g., `[grasp]`).

#### Steps to Complete:

1. **Set up the target**: Define the target object (e.g., `object_designator`).
2. **Resolve the robot**: Use `robot_desig.resolve()` to specify which robot instance is interacting.
3. **Choose an arm**: Select the arm you want to use for this task.
4. **Use the determined grasp**: Pass in the appropriate grasp configuration, stored in `grasp = calculate_object_faces(self.object_designator)[0]`.

#### Code Template

Complete the following code to find a suitable pose with `CostmapLocation`:

```python
# Define the location using CostmapLocation
pickup_loc = CostmapLocation(
    target=____,  # Use the object designator
    reachable_for=____,  # Resolve the robot designator
    reachable_arm=____,  # Specify the arm you are using
    used_grasps=____  # Use the determined grasp in a list
)

In [ ]:
## add your code here

<details>

<summary>Click here for the Solution.</summary>

```python
with simulated_robot:
   grasp = calculate_object_faces(self.object_designator)[0]

        pickup_loc = CostmapLocation(
            target=self.object_designator,
            reachable_for=robot_desig.resolve(),
            reachable_arm=self.arm,
            used_grasps=[grasp])
```
</details>
You will end up with a list of possible pickup locations. Now, we’ll iterate through these poses to check if any are reachable by the specified arm.

```python
pickup_pose = next((pose for pose in pickup_loc if self.arm in pose.reachable_arms), None)
if not pickup_pose:
    raise ObjectUnfetchable(
        f"No reachable pose found for the robot to grasp the object: {self.object_designator} with arm: {self.arm}"
    ) 

Once a reachable pose is found, navigate to it and perform a PickUpAction. This action requires the object designator, the selected arm, and the previously determined grasp as parameters.

In [ ]:
## add your code here


<details>

<summary>Click here for the Solution.</summary>

```python
with simulated_robot:

    PickUpAction(spoon_desig, [Arms.LEFT], [Grasp.TOP]).resolve().perform()
```
</details>


## Step 5: Transport the Object and Final Placement

In this step, we’ll use the calculated pose to pick up the milk and place it at the target location.

*Objective:* By the end of this step, you will have executed a full pick-and-place operation.


### Exercise: Write the plan
Your task now is to write a complete plan that includes transporting the object and closing the fridge. The target pose for the milk is:

```python
milk_target_pose = Pose([5.34, 3.55, 0.8])
```

In [ ]:
## add your code here


<details>

<summary>Click here for the Solution.</summary>

```python
with simulated_robot:
    poseHard = Pose([1.3, 2.7, 0], [0, 0, 1, 0])
    NavigateAction([poseHard]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([TorsoState.HIGH]).resolve().perform()
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    closed_location, opened_location = AccessingLocation(handle_desig=handle_desig.resolve(),
                                                         robot_desig=robot_desig.resolve()).resolve()
    OpenAction(object_designator_description=handle_desig, arms=[closed_location.arms[0]],
               start_goal_location=[closed_location, opened_location]).resolve().perform()

    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    obj_desig = DetectAction(milk_desig).resolve().perform()
    milk_target_pose = Pose([5.34, 3.55, 0.8])
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([TorsoState.HIGH]).resolve().perform()
     robot_desig = BelieveObject(names=[RobotDescription.current_robot_description.name])
        ParkArmsActionPerformable(Arms.BOTH).perform()

        if self.object_designator.obj_type == ObjectType.BOWL or self.object_designator.obj_type == ObjectType.SPOON:
            grasp = calculate_object_faces(self.object_designator)[1]
        else:
            grasp = calculate_object_faces(self.object_designator)[0]

        pickup_loc = CostmapLocation(
            target=self.object_designator,
            reachable_for=robot_desig.resolve(),
            reachable_arm=self.arm,
            used_grasps=[grasp]
        )

        # Tries to find a pick-up posotion for the robot that uses the given arm
        pickup_pose = next((pose for pose in pickup_loc if self.arm in pose.reachable_arms), None)
        if not pickup_pose:
            raise ObjectUnfetchable(
                f"No reachable pose found for the robot to grasp the object: {self.object_designator} with arm: {self.arm}"
            )

        NavigateActionPerformable(pickup_pose.pose).perform()
        PickUpActionPerformable(self.object_designator, self.arm, grasp).perform()
        ParkArmsActionPerformable(Arms.BOTH).perform()
        try:
            place_loc = CostmapLocation(
                target=self.target_location,
                reachable_for=robot_desig.resolve(),
                reachable_arm=self.arm,
                used_grasps=[grasp],
                object_in_hand=self.object_designator
            ).resolve()
        except StopIteration:
            raise ReachabilityFailure(
                f"No reachable location found for the target location: {self.target_location}"
            )
        NavigateActionPerformable(place_loc.pose).perform()
        PlaceActionPerformable(self.object_designator, self.arm, self.target_location).perform()
        ParkArmsActionPerformable(Arms.BOTH).perform()
```
</details>


Now we have our first complete transport plan! While this is a solid foundation for understanding the basics, it lacks some advanced cognitive capabilities.

### What’s Missing?

- **Failure Handling**: Adding failure handling is crucial. It not only makes the robot more resilient but also highlights the complexities a robot control system must navigate. Designing a generalized failure-handling approach requires the control program to be semantically transparent, allowing it to retry, replan, transform, or adapt to different types of failures without hidden information.

- **Learning from Experience**: Another concept is memory-based learning. By logging each action the robot performs, we can replay and analyze this data to help the robot identify and correct mistakes. You’ll find more on this in Chapter 6’s extra materials.

- **Enhanced Physical Understanding**: In this simulation, the robot's physical interactions are simplified, which bypasses some real-world challenges. Implementing true physical understanding would involve additional considerations beyond what’s covered here.
